# aDDM_gridSearch Script
## Using z-scored values

Implement the iterative step for the search of optimal parameter sets. Use of aDDM_pta_mla script from addm_toolbox (https://github.com/goptavares/aDDM-Toolbox)

In [1]:
from __future__ import absolute_import
import argparse
import addm_toolbox 
import numpy as np
import pandas as pd
import os

In [2]:
addm_toolbox

<module 'addm_toolbox' from '/Users/pradyumna/anaconda/lib/python3.5/site-packages/addm_toolbox/__init__.py'>

In [3]:
addm_toolbox.addm_pta_mle_cont

<module 'addm_toolbox.addm_pta_mle_cont' from '/Users/pradyumna/anaconda/lib/python3.5/site-packages/addm_toolbox/addm_pta_mle_cont.py'>

### + Free parameters in the model

#####    d : controls speed of signal integration (Free DDM parameter)
#####   sigma: standard deviation of normal distribution (Free DDM parameter)
#####   theta: controls attentional bias (Free aDDM parameter)

### + Inputs for toolobox addm_pta_mle script 
##### range_d,range_sigma,range_theta : range to define the grid for parameter search
##### trials_file_name : filepath with trial conditions (to be used for simulations)
##### expdata_file_name :  filepath with trial info [participant, trial number, RT, response, values]
##### fixations_file_name : filepath with fixations info [participant, trial num, fixation_id, fixation_time]
##### Extra parameters: trials_per_subject, simulations_per_condition, subject_ids (number of the subjects to be included for adjusting the parameters, num_threads (parallel processing),save_simulations, save_figures, verbose (deliver reports)

###  + Initial grid selection:
##### range_d = [0.003, 0.006, 0.009] ; range_sigma = [0.03, 0.06, 0.09] ; range_theta = [0.3, 0.5, 0.7]

### + Iteration (adjustment to a new grid) 

#### ** aDDM grid iterations using version of the script that does not transform item values **

## LESS INV condition

In [ ]:
condition_type_name = "Less_zScore_32"
toSaveFolder = 'aDDM_simulations/'+condition_type_name+'/'

iteration_data = []
current_min_nll = 0
previous_min_nll = 0

# Initial grid selection
#range_d = [0.003, 0.006, 0.009]
#range_sigma = [0.03, 0.06, 0.09]
#range_theta = [0.3, 0.5, 0.7]

range_d = [0.001, 0.002,0.003]
range_sigma = [0.01,0.03 ,0.05]
range_theta = [0.6, 0.7,0.8]


#data files definition
trials_file_name = '/Users/pradyumna/Documents/GiTs/PF2019/InputData_aDDM/trial_conditions_parcode_zScore_test.csv'
#expdata_file_name = '/Users/bdmlab/OneDrive - University College London/PhD Projects/GiTs/FF2018/InputData/expdata_FFA_Dislike_v2.csv'
#fixations_file_name = '/Users/bdmlab/OneDrive - University College London/PhD Projects/GiTs/FF2018/InputData/fixations_FFA_Dislike_v2.csv'
expdata_file_name = '/Users/pradyumna/Documents/GiTs/PF2019/InputData_aDDM/aDDM_expdata_PF2019_Less_zScore_32.csv'
fixations_file_name = '/Users/pradyumna/Documents/GiTs/PF2019/InputData_aDDM/aDDM_Fix_PF2019_Less_NoBin_33.csv'

trials_per_subject = 120
simulations_per_condition = 400

iter_num = 1

subjectIds=[1] ## All participants


#Add valueDiffs for plotting
# For unbinned case
#l = range(-300,301,5)
#l = [x * 0.01 for x in l]

# For bin case
l = np.arange(-6,7,1)


#Give a name to the folder to store the simulations
if not os.path.exists('aDDM_simulations/'+condition_type_name):
    os.makedirs('aDDM_simulations/'+condition_type_name)

while ((abs(current_min_nll-previous_min_nll) > (previous_min_nll*0.05/100)) and ((current_min_nll-previous_min_nll)>0))  or iter_num == 1:
    
    if iter_num > 1:  
        delta_d = range_d[1]-range_d[0]
        delta_sigma = range_sigma[1] - range_sigma[0]
        delta_theta = range_theta[1] - range_theta[0]
        
        range_d = [new_d - delta_d/2 , new_d ,new_d + delta_d/2]
        range_sigma =[new_sigma - delta_sigma/2 , new_sigma ,new_sigma + delta_sigma/2]
        range_theta =[new_theta - delta_theta/2 , new_theta ,new_theta + delta_theta/2]
    
    
    # control for the limits 0 < theta < 1 
    if range_theta[0]<0:
        range_theta[0] = 0
        
    if range_theta[2]>1:
        range_theta[0] = 1
    
    # control for the limits 0 < d 
    if range_d[0]<0:
        range_d[0] = 0

    
    
 #run the toolbox
    params, NLL = addm_toolbox.addm_pta_mle_cont.main(range_d, range_sigma, range_theta,
                       trials_file_name, 
                       expdata_file_name,
                       fixations_file_name, 
                       trials_per_subject,simulations_per_condition, subjectIds,
                       9,saveSimulations=True, saveFigures=True,
                       verbose = True)
    
    
    
    # Update the NLL values for this iteration
    previous_min_nll = current_min_nll
    current_min_nll = NLL
    
    #get the new values
    new_d = params[0]
    new_sigma = params[1]
    new_theta = params[2]
    
    iteration_data.append([iter_num,new_d, new_sigma, new_theta,current_min_nll]) 

    iter_num = iter_num + 1

    
iteration_dataf=pd.DataFrame(iteration_data,columns=['Iteration', 'd', 'sigma', 'theta', 'min_NLL'])
print ("Finished iterations!!")


Loading experimental data...
Starting grid search...
Computing likelihoods for model (0.001, 0.01, 0.6)...
Computing likelihoods for model (0.001, 0.01, 0.7)...
Computing likelihoods for model (0.001, 0.01, 0.8)...
Computing likelihoods for model (0.001, 0.03, 0.6)...
Computing likelihoods for model (0.001, 0.03, 0.7)...
Computing likelihoods for model (0.001, 0.03, 0.8)...
Computing likelihoods for model (0.001, 0.05, 0.6)...
Computing likelihoods for model (0.001, 0.05, 0.7)...
Computing likelihoods for model (0.001, 0.05, 0.8)...
Computing likelihoods for model (0.002, 0.01, 0.6)...
Computing likelihoods for model (0.002, 0.01, 0.7)...
Computing likelihoods for model (0.002, 0.01, 0.8)...
Computing likelihoods for model (0.002, 0.03, 0.6)...
Computing likelihoods for model (0.002, 0.03, 0.7)...
Computing likelihoods for model (0.002, 0.03, 0.8)...
Computing likelihoods for model (0.002, 0.05, 0.6)...
Computing likelihoods for model (0.002, 0.05, 0.7)...
Computing likelihoods for mod

In [5]:
iteration_dataf.to_csv(toSaveFolder + 'ParameterIterations.csv',index = True)

In [6]:
iteration_dataf

,Iteration,d,sigma,theta,min_NLL
0,1,0.001,0.05,0.01,13826.983475
1,2,0.001,0.05,0.01,13826.983475


*** 

In [ ]:
### 